In [ ]:
pip install shap

     |████████████████████████████████| 564 kB 8.8 MB/s 


## Setup

In [2]:
from google.colab import drive
drive.mount('drive')
# Import libraries
import pandas as pd
import numpy as np

Mounted at drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/train.csv')
X_train = train.drop("change_lead", axis = 1)
y_train = train["change_lead"]

In [4]:
test = pd.read_csv('/content/drive/MyDrive/Data Modelling/Stock Prices Prediction/test.csv')
X_test = test.drop("change_lead", axis = 1)
y_test = test["change_lead"]

In [5]:
# Seed value
seed_value= 2022

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

## Helper Function 

In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr
from sklearn import preprocessing
from scipy.stats import chi2_contingency
from sklearn import preprocessing
from scipy import stats
from sklearn import tree, ensemble, model_selection, metrics
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
# import shap
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from datetime import datetime
import tensorflow as tf
import keras

from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb

In [25]:
from sklearn import metrics
agg_accuracy_score_dict = {}
agg_f1_score_dict={}
agg_roc_auc_dict={}

train_accuracy_score_dict = {}
train_f1_score_dict={}
train_roc_auc_dict={}

val_accuracy_score_dict = {}
val_f1_score_dict={}
val_roc_auc_dict={}

test_accuracy_score_dict = {}
test_f1_score_dict={}
test_roc_auc_dict={}

def evaluation(true, pred, title):
    print("================== Evaluation on {} ==================".format(title))
    # accuracy 
    acc = accuracy_score(true, pred)
    print("accuracy: {}\n".format(acc))
    agg_accuracy_score_dict[title] = acc
    # precision
    pre = precision_score(true, pred)
    print(f"precision: {pre}\n")
    # f1 score
    print("classification report: \n{}\n".format(classification_report(true, pred)))
    print("F1 score: \n{}\n".format(metrics.f1_score(true, pred)))
    f1 = f1_score(true, pred)
    agg_f1_score_dict[title] = f1
    # confusion matrix
    print("Confusion matrix: \n{}\n".format(confusion_matrix(true, pred)))
    # roc auc result
    fpr, tpr, thresholds = roc_curve(true, pred)
    roc_auc = auc(fpr, tpr)
    print("ROC AUC: {}\n".format(roc_auc))
    agg_roc_auc_dict[title] = roc_auc
    print("================== End of Evaluation on {} ==================".format(title))
    
    if "train dataset" in title:
        train_accuracy_score_dict[title] = acc
        train_f1_score_dict[title] = f1
        train_roc_auc_dict[title] = roc_auc
    elif "test dataset" in title:
        test_accuracy_score_dict[title] = acc
        test_f1_score_dict[title] = f1
        test_roc_auc_dict[title] = roc_auc
    else:
        val_accuracy_score_dict[title] = acc
        val_f1_score_dict[title] = f1
        val_roc_auc_dict[title] = roc_auc

## SVM

### Default Parameters

In [26]:
#Import svm model
from sklearn import svm
from sklearn import metrics
#Create a svm Classifier

svm = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
svm.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_train = svm.predict(X_train)

# Get the training accuracy
# print('Training F1: ', metrics.f1_score(y_train, svm.predict(X_train)))
# print the accuracy of its predictions
# metrics.f1_score(y_test, y_pred_train)
# metrics.accuracy_score(y_test, y_pred_train)

# metrics.precision_score(y_test, y_pred_train)
evaluation(y_train, y_pred_train, "Default SVM on train dataset")

================== Evaluation on Default SVM on train dataset ==================
accuracy: 0.528415961305925

precision: 0.5494137353433836

classification report: 
              precision    recall  f1-score   support

           0       0.51      0.55      0.53      1193
           1       0.55      0.51      0.53      1288

    accuracy                           0.53      2481
   macro avg       0.53      0.53      0.53      2481
weighted avg       0.53      0.53      0.53      2481


F1 score: 
0.5286059629331183

Confusion matrix: 
[[655 538]
 [632 656]]

ROC AUC: 0.5291764068869648

================== End of Evaluation on Default SVM on train dataset ==================


In [27]:
y_pred_test = svm.predict(X_test)

# Get the testing accuracy
# print('testing F1: ', metrics.f1_score(y_test, svm.predict(X_test)))
# print the accuracy of its predictions
# print('Test F1: ', metrics.f1_score(y_test, y_pred))
# print('accuracy:', metrics.accuracy_score(y_test, y_pred)
evaluation(y_test, y_pred_test, "Default SVM on test dataset")

================== Evaluation on Default SVM on test dataset ==================
accuracy: 0.4782608695652174

precision: 0.4489795918367347

classification report: 
              precision    recall  f1-score   support

           0       0.51      0.45      0.48       147
           1       0.45      0.51      0.48       129

    accuracy                           0.48       276
   macro avg       0.48      0.48      0.48       276
weighted avg       0.48      0.48      0.48       276


F1 score: 
0.47826086956521735

Confusion matrix: 
[[66 81]
 [63 66]]

ROC AUC: 0.4803037494067395

================== End of Evaluation on Default SVM on test dataset ==================


## Hyperparameter Tuning



*   Randomized Search



In [28]:
from sklearn.model_selection import RandomizedSearchCV
# Values for C:
C = [0.001, 0.01, 0.1, 1, 10, 100] # works
# C = [0.001, 0.01, 0.1, 1, 10]
break_ties = False
cache_size = 200,
# class_weight = [1,1,1,1,1,1]
l1 = np.arange(0,1,0.01)
l2 = np.arange(0,1,0.01)
class_weight = [{0:i,1:j} for i,j in zip(l1,l2)]
coef0 = 0.0,
decision_function_shape = 'ovr'
degree = [1,2,3,4,5]
# gamma = 'scale'
gamma = np.logspace(-9, 3, 13) # works
# gamma = np.logspace(-7, 5, 13)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
max_iter = -1
probability = [True, False]
random_state = None
shrinking = [True, False]
tol = 0.001
verbose = [True, False]
# Create the random grid
random_grid = {'C': C,
              # 'break_ties': break_ties,
              # 'cache_size': cache_size,
              'class_weight': class_weight,
              # 'coef0': coef0,
              # 'decision_function_shape': decision_function_shape,
              # 'degree': degree,
              'gamma': gamma,
              'kernel': kernel,
              # 'max_iter': max_iter,
              # 'probability': probability,
              # 'random_state': random_state,
              # 'shrinking': shrinking,
              # 'tol': tol,
              # 'verbose': verbose
               }
# pprint(random_grid)

In [32]:
# Use the random grid to search for best hyperparameters
from sklearn import svm
# First create the base model to tune
svm = svm.SVC()

# Perform Randomised Search
svm_random = RandomizedSearchCV(estimator = svm, param_distributions = random_grid)

# Fit the random search model
svm_random.fit(X_train, y_train)
svm_random.best_params_
best_estimators = svm_random.best_estimator_

In [33]:
best_estimators

SVC(C=10, class_weight={0: 0.44, 1: 0.44}, gamma=1e-05, kernel='poly')

In [35]:
from sklearn import svm
best_estimators.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_test = best_estimators.predict(X_test)

# Evaluate the model
evaluation(y_test, y_pred_test, "Default SVM on train dataset")

================== Evaluation on Default SVM on train dataset ==================
accuracy: 0.4673913043478261

precision: 0.4673913043478261

classification report: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       147
           1       0.47      1.00      0.64       129

    accuracy                           0.47       276
   macro avg       0.23      0.50      0.32       276
weighted avg       0.22      0.47      0.30       276


F1 score: 
0.6370370370370371

Confusion matrix: 
[[  0 147]
 [  0 129]]

ROC AUC: 0.5

================== End of Evaluation on Default SVM on train dataset ==================


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
